In [175]:
import requests
import pandas as pd
import private

# User Variables
ETHERSCAN_API_KEY = private.etherscan_api
CONTRACT_ADDRESS = "0x827922686190790b37229fd06084350E74485b72"
ETHERSCAN_V2_API = 'https://api.etherscan.io/v2/api'
FILTER_ADDRESSES = [private.wal_lp.lower(), private.sickle_lp.lower()]
METHOD_ID_MAP = {
    "0x2812d614": "Compound",
    "0xf5304377": "Deposit",
    "0x5ec5999e": "Harvest",
    "0xe6fb317f": "RebalanceFor",
    "0xe5bacdd0": "Increase",
    "0x1c396db6": "Decrease",
    "0x28734381": "Exit",
    "0x659b91b1": "Rebalance",
    "0x22451262": "Move"
}
TOPIC_ID = "0xbf9d03ac543e8f596c6f4af5ab5e75f366a57d2d6c28d2ff9c024bd3f88e8771"

In [53]:
# Function to fetch NFT transfers
def get_transactions(start_block, api_key, address, action='tokentx'):
    # Just pull for given token IDs (based on Contract Address)
        params = {
            'chainid': '8453',  # base mainnet = 8453
            'module': 'account',
            'action': action,
            # 'contractaddress': contract_address,
            'address': address,
            'page': '1',
            'offset': '10000',
            'startblock': start_block,
            'endblock': 99999999,
            'sort': 'asc',
            'apikey': api_key
        }

        response = requests.get(f'{ETHERSCAN_V2_API}', params=params)
        data = response.json()
        if data["status"] != "1":
            print("Error fetching data:", data["message"], data['result'])
            return []
        
        return data["result"]

In [73]:
nft_transactions = get_transactions(25707200, private.etherscan_api, private.sickle_lp, 'tokennfttx')
token_transactions = get_transactions(25707200, private.etherscan_api, private.sickle_lp, 'tokentx')

In [ ]:
# Function to fetch transaction details and extract method ID
def fetch_transaction_details(tx_hash, api_key):
    url = f"{ETHERSCAN_V2_API}?chainid=8453&module=proxy&action=eth_getTransactionByHash&txhash={tx_hash}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if "result" in data and "input" in data["result"]:
        method_id = data["result"]["input"][:10]  # First 10 chars are the method ID
        token_id_pos = 64 * 3 + 10
        token_id = int(data["result"]["input"][token_id_pos:token_id_pos + 64], 16)
        x = METHOD_ID_MAP.get(method_id.lower(), "Unknown")
        print(f"{x} occured at tx {tx_hash[:6]}...{tx_hash[-4:]}")
        return x, token_id
    return "Unknown"

In [ ]:
x, y = fetch_transaction_details("0x8c3e050eb6dd997be5556c4524e2e7646248b66bde45607bb017860da6ebe81c", private.etherscan_api)
print(x)

In [224]:
def process_token_transfers(transfers, nft_df):
    df = pd.DataFrame(transfers)
    df = df[["blockNumber", "timeStamp", "hash", "from", "to", "value", "tokenSymbol", "tokenDecimal"]]
    df["timeStamp"] = pd.to_datetime(pd.to_numeric(df["timeStamp"], errors='coerce'), unit='s')
    df["value"] = df["value"].astype(float) / (10 ** df["tokenDecimal"].astype(int))  # Convert to standard token value
    
    # Filter only transactions where both 'from' and 'to' are in FILTER_ADDRESSES
    df = df[df["from"].isin(FILTER_ADDRESSES) & df["to"].isin(FILTER_ADDRESSES)]
    
    # Link token transfers to NFT burn and mint transactions
    df = df.merge(nft_df[['hash', 'seriesID', 'eventType', 'tokenID']], on='hash', how='left')

    # Remove duplicate token transfers within the same transaction hash
    df = df.drop_duplicates(subset=["hash", "from", "to", "value", "tokenSymbol"])

    # Assign method ID event type for missing values
    # Get a list of hashes that don't have an event type yet
    tx_hashes = df[pd.isna(df["eventType"])]["hash"].unique()
    # Fetch the event type for each hash
    event_types = {}
    token_ids = {}
    for hash in tx_hashes:
        event_types[hash], token_ids[hash] = fetch_transaction_details(hash, ETHERSCAN_API_KEY)
        
    # Apply the event type to the data frame by matching the hash value
    df["eventType"] = df.apply(lambda row: event_types.get(row["hash"], row["eventType"]), axis=1)
    df["tokenID"] = df.apply(lambda row: token_ids.get(row["hash"], row["tokenID"]), axis=1)
    df["tokenID"] = df["tokenID"].astype(int)

    # Fill missing seriesID using tokenID mapping
    token_to_series = df.dropna(subset=["seriesID"]).set_index("tokenID")["seriesID"].to_dict()
    df["seriesID"] = df["seriesID"].fillna(df["tokenID"].map(token_to_series))
    
    df.sort_values(by=["seriesID", "timeStamp"], inplace=True)
    
    return df

In [205]:
# Function to process transactions and track series
def process_transactions(transactions, token_transfers):
    df = pd.DataFrame(transactions)
    df = df[["blockNumber", "timeStamp", "hash", "from", "to", "tokenID"]]
    df["timeStamp"] = pd.to_datetime(pd.to_numeric(df["timeStamp"], errors='coerce'), unit='s')
    
    # Identify burns (to burn address) and mints (from address is zero address)
    burn_address = "0x0000000000000000000000000000000000000000"
    df = df[df["from"].eq(burn_address) | df["to"].eq(burn_address)]  # only care about mint and burn, not transfers to the Aerodrome farms
    df["eventType"] = df["from"].apply(lambda x: "Mint" if x == burn_address else "Burn")
    
    # Identify NFT series
    series_map = {}
    token_df = pd.DataFrame(token_transfers)
    for _, row in token_df.iterrows():
        if row["from"].lower() == private.wal_lp.lower():
            tx_hash = row["hash"]
            # Extract rows from token_df with the same hash value
            subset = token_df[token_df["hash"] == tx_hash]
            tokens = subset["tokenSymbol"].tolist()
            # Create string out of the first two items in tokens list
            seriesID = '/'.join(map(str, tokens[:2])) + '_' + tx_hash[-4:]
            matching_nft_mints = df[(df["hash"] == tx_hash) & (df["eventType"] == "Mint")]
            for _, mint in matching_nft_mints.iterrows():
                series_map[mint["tokenID"]] = seriesID
    
    # Assign series ID to subsequent burns and mints
    df["seriesID"] = df["tokenID"].map(series_map)
    for _, row in df.iterrows():
        if row["eventType"] == "Burn" and row["tokenID"] in series_map:
            tx_hash = row["hash"]
            new_mint = df[(df["hash"] == tx_hash) & (df["eventType"] == "Mint")]
            for _, mint in new_mint.iterrows():
                series_map[mint["tokenID"]] = series_map[row["tokenID"]]

    # Get unique list of all the transaction hash values
    tx_hashes = df["hash"].unique()
    # Fetch the event type for each hash
    event_types = {}
    for hash in tx_hashes:
        event_types[hash], id = fetch_transaction_details(hash, ETHERSCAN_API_KEY)
    # Apply the event type to the data frame by matching the hash value
    df["eventType"] = df["hash"].map(event_types)

    df["seriesID"] = df["tokenID"].map(series_map)
    df.sort_values(by=["seriesID", "timeStamp"], inplace=True)
    return df

In [ ]:
nft = process_transactions(nft_transactions, token_transactions)

In [225]:
tokens = process_token_transfers(token_transactions, nft)

Harvest occured at tx 0x64c7...bd0e
Harvest occured at tx 0x3083...4dc9
Harvest occured at tx 0x49da...dcec
Harvest occured at tx 0xb69d...6d35
Compound occured at tx 0x8584...2d99
Harvest occured at tx 0xf7b3...5322
Harvest occured at tx 0xbe32...1dc1
Harvest occured at tx 0xcc40...363b
Harvest occured at tx 0xf05a...f773
Increase occured at tx 0x8c3e...e81c
Harvest occured at tx 0xd324...1fac
